<a href="https://colab.research.google.com/github/shonyeajin/KCBD/blob/main/%EC%BC%80%EB%9D%BC%EC%8A%A4_%EC%BD%9C%EB%B0%B1%EA%B3%BC_%ED%85%90%EC%84%9C%EB%B3%B4%EB%93%9C%EB%A5%BC_%EC%82%AC%EC%9A%A9%ED%95%98%EC%97%AC_%EA%B2%80%EC%82%AC%EC%99%80_%EB%AA%A8%EB%8B%88%ED%84%B0%EB%A7%81.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ModelCheckPoint 와 EarlyStopping 콜백


In [ ]:
import keras

callbacks_list=[
                keras.callbacks.EarlyStopping(
                    monitor='val_acc',
                    patience=1,
                ),
                keras.callbacks.ModelCheckpoint(
                    filepath='my_model.h5',
                    monitor='val_loss',
                    save_best_only=True,
                )
]
model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
model.fit(x,y,
          epochs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

ReduceLROnPlateau 콜백


In [ ]:
callbacks_list=[
               keras.callback.ReduceLROnPlateau(
                   monitor='val_loss',
                   factor=0.1,
                   patience=10,
               )
]

model.fit(x,y,
          epcohs=10,
          batch_size=32,
          callbacks=callbacks_list,
          validation_data=(x_val, y_val))

자신의 콜백 만들기
(매 에포크의 끝에서 검증 세트의 첫 번쨰 샘플로 모델이 있는 모든 층이 활성화 출력을 계산하여 (넘파이 배열로) 디스크레 저장하는 자작 콜백의 예)

In [ ]:
import keras
import numpy as np

class ActivationLogger(keras.callbacks.Callback):
  def set_model(self, model):
    self.model=model
    layers_outputs=[layer.output for layer in model.layers]
    self.activations_model=keras.models.Model(model.input, layer_outputs)

  def on_epoch_end(self, epoch, logs=None):
    if self.validation_data is None:
      raise RuntimeError('Requires validation_data.')

    validation_sample=self.validation_data[0][0:1]
    activations=self.activations_model.predict(validation_sample)
    f=open('activations_at_epoch_'+str(epoch)+'.npz', 'wb')
    np.savez(f, activations)
    f.close()

텐서보드를 사용한 텍스트 분류 모델

In [1]:
import keras
from keras import layers
from keras.datasets import imdb
from keras.preprocessing import sequence

max_features=2000
max_len=500

(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
x_train=sequence.pad_sequences(x_train, maxlen=max_len)
x_test=sequence.pad_sequences(x_test, maxlen=max_len)

model= keras.models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len, name='embed'))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPooling1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPooling1D())
model.add(layers.Dense(1))
model.summary()
model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['acc'])

17473536/17464789 [==============================] - 2s 0us/step
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed (Embedding)           (None, 500, 128)          256000    
                                                                 
 conv1d (Conv1D)             (None, 494, 32)           28704     
                                                                 
 max_pooling1d (MaxPooling1D  (None, 98, 32)           0         
 )                                                               
                                                                 
 conv1d_1 (Conv1D)           (None, 92, 32)            7200      
                                                                 
 global_max_pooling1d (Globa  (None, 32)               0         
 lMaxPooling1D)                                                  
                                                         

In [2]:
!mkdir my_log_dir

In [4]:
callbacks=[
           keras.callbacks.TensorBoard(
               log_dir='my_log_dir',
               histogram_freq=1,
               embeddings_freq=1,
           )
]
history=model.fit(x_train, y_train, epochs=20,
                  batch_size=128,
                  validation_split=0.2,
                  callbacks=callbacks)

Epoch 1/20
157/157 [==============================] - 13s 24ms/step - loss: 0.5810 - acc: 0.6883 - val_loss: 0.4315 - val_acc: 0.8300
Epoch 2/20
157/157 [==============================] - 3s 21ms/step - loss: 0.4299 - acc: 0.8541 - val_loss: 0.4230 - val_acc: 0.8582
Epoch 3/20
157/157 [==============================] - 3s 22ms/step - loss: 0.3631 - acc: 0.8867 - val_loss: 0.4650 - val_acc: 0.8666
Epoch 4/20
157/157 [==============================] - 3s 21ms/step - loss: 0.3192 - acc: 0.9071 - val_loss: 0.4930 - val_acc: 0.8676
Epoch 5/20
157/157 [==============================] - 3s 22ms/step - loss: 0.2706 - acc: 0.9280 - val_loss: 0.5727 - val_acc: 0.8678
Epoch 6/20
157/157 [==============================] - 3s 21ms/step - loss: 0.2414 - acc: 0.9419 - val_loss: 0.6522 - val_acc: 0.8670
Epoch 7/20
157/157 [==============================] - 3s 22ms/step - loss: 0.2085 - acc: 0.9590 - val_loss: 0.7278 - val_acc: 0.8692
Epoch 8/20
157/157 [==============================] - 3s 21ms/step -

In [5]:
!tensorboard --logdir=my_log_dir


NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.7.0 at http://localhost:6006/ (Press CTRL+C to quit)
^C
